In [1168]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import SGDRegressor, LinearRegression, Ridge
from sklearn import metrics
%matplotlib inline

import warnings
def ignore_warn():
    pass
warnings.warn = ignore_warn

In [1169]:
data = pd.read_csv('./House_Rent_Dataset.csv')
data.head()

,Posted On,BHK,Rent,Size,Floor,Area Type,Area Locality,City,Furnishing Status,Tenant Preferred,Bathroom,Point of Contact
0,2022-05-18,2,10000,1100,Ground out of 2,Super Area,Bandel,Kolkata,Unfurnished,Bachelors/Family,2,Contact Owner
1,2022-05-13,2,20000,800,1 out of 3,Super Area,"Phool Bagan, Kankurgachi",Kolkata,Semi-Furnished,Bachelors/Family,1,Contact Owner
2,2022-05-16,2,17000,1000,1 out of 3,Super Area,Salt Lake City Sector 2,Kolkata,Semi-Furnished,Bachelors/Family,1,Contact Owner
3,2022-07-04,2,10000,800,1 out of 2,Super Area,Dumdum Park,Kolkata,Unfurnished,Bachelors/Family,1,Contact Owner
4,2022-05-09,2,7500,850,1 out of 2,Carpet Area,South Dum Dum,Kolkata,Unfurnished,Bachelors,1,Contact Owner


### Предварительная обработка данных
В первую очередь проверим есть ли пропущенные значения

In [1170]:
data.isna().sum()

Posted On            0
BHK                  0
Rent                 0
Size                 0
Floor                0
Area Type            0
Area Locality        0
City                 0
Furnishing Status    0
Tenant Preferred     0
Bathroom             0
Point of Contact     0
dtype: int64

Пропущенных значений нет

In [1171]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4746 entries, 0 to 4745
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Posted On          4746 non-null   object
 1   BHK                4746 non-null   int64 
 2   Rent               4746 non-null   int64 
 3   Size               4746 non-null   int64 
 4   Floor              4746 non-null   object
 5   Area Type          4746 non-null   object
 6   Area Locality      4746 non-null   object
 7   City               4746 non-null   object
 8   Furnishing Status  4746 non-null   object
 9   Tenant Preferred   4746 non-null   object
 10  Bathroom           4746 non-null   int64 
 11  Point of Contact   4746 non-null   object
dtypes: int64(4), object(8)
memory usage: 445.1+ KB


In [1172]:
# Посмотрим на количество уникальных значений в каждом из столбцов
[(data.columns[i], data.iloc[:, i].unique().shape[0]) for i in range(len(data.columns))]

[('Posted On', 81),
 ('BHK', 6),
 ('Rent', 243),
 ('Size', 615),
 ('Floor', 480),
 ('Area Type', 3),
 ('Area Locality', 2235),
 ('City', 6),
 ('Furnishing Status', 3),
 ('Tenant Preferred', 3),
 ('Bathroom', 8),
 ('Point of Contact', 3)]

`Area Locality` содержит слишком много уникальных значений.

In [1173]:
data = data.drop(['Area Locality'], axis = 1)

Так как `Floor`` де-факто содержит два параметра: 
1. общее количество этажей в доме
2. этаж, на котором располагаются аппартаменты,

то эту строку стоит разделить на отдельные составляющие

In [1174]:
data["Total Floors"] = data["Floor"].apply(lambda floor:floor.split()[-1])
data["Floor"] = data["Floor"].apply(lambda floor:floor.split()[0])
encodes = {"Floor":     {"Ground": 0, "Upper": -1, "Lower": -2},
           "Total Floors": {"Ground": 0}}

data = data.replace(encodes)
data.head()

,Posted On,BHK,Rent,Size,Floor,Area Type,City,Furnishing Status,Tenant Preferred,Bathroom,Point of Contact,Total Floors
0,2022-05-18,2,10000,1100,0,Super Area,Kolkata,Unfurnished,Bachelors/Family,2,Contact Owner,2
1,2022-05-13,2,20000,800,1,Super Area,Kolkata,Semi-Furnished,Bachelors/Family,1,Contact Owner,3
2,2022-05-16,2,17000,1000,1,Super Area,Kolkata,Semi-Furnished,Bachelors/Family,1,Contact Owner,3
3,2022-07-04,2,10000,800,1,Super Area,Kolkata,Unfurnished,Bachelors/Family,1,Contact Owner,2
4,2022-05-09,2,7500,850,1,Carpet Area,Kolkata,Unfurnished,Bachelors,1,Contact Owner,2


Поценциально цена аренды может зависить от даты публикации объявления, т.к. в зависимости от экономической ситуации и других факторов цена может меняться, но в данном случае этот значения находятся в пределах 3 месяцев, тем более мы работаем с линейной регрессией, а не с последовательными данными.

Данные о дате публикации объявления `Posted On` возможно разбить на составляющие: день, месяц, день недели и пр., но это тоже имеет мало смысла.

Параметр нам не полезен, поэтому избавимся от него.

In [1175]:
data = data.drop('Posted On', axis = 1)
data.head()

,BHK,Rent,Size,Floor,Area Type,City,Furnishing Status,Tenant Preferred,Bathroom,Point of Contact,Total Floors
0,2,10000,1100,0,Super Area,Kolkata,Unfurnished,Bachelors/Family,2,Contact Owner,2
1,2,20000,800,1,Super Area,Kolkata,Semi-Furnished,Bachelors/Family,1,Contact Owner,3
2,2,17000,1000,1,Super Area,Kolkata,Semi-Furnished,Bachelors/Family,1,Contact Owner,3
3,2,10000,800,1,Super Area,Kolkata,Unfurnished,Bachelors/Family,1,Contact Owner,2
4,2,7500,850,1,Carpet Area,Kolkata,Unfurnished,Bachelors,1,Contact Owner,2


In [1176]:
columns = ['Area Type', 'City', 'Furnishing Status', 'Tenant Preferred', 'Point of Contact']
data = pd.get_dummies(data, columns=columns, dtype='int')

data.info()
data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4746 entries, 0 to 4745
Data columns (total 24 columns):
 #   Column                             Non-Null Count  Dtype 
---  ------                             --------------  ----- 
 0   BHK                                4746 non-null   int64 
 1   Rent                               4746 non-null   int64 
 2   Size                               4746 non-null   int64 
 3   Floor                              4746 non-null   object
 4   Bathroom                           4746 non-null   int64 
 5   Total Floors                       4746 non-null   object
 6   Area Type_Built Area               4746 non-null   int32 
 7   Area Type_Carpet Area              4746 non-null   int32 
 8   Area Type_Super Area               4746 non-null   int32 
 9   City_Bangalore                     4746 non-null   int32 
 10  City_Chennai                       4746 non-null   int32 
 11  City_Delhi                         4746 non-null   int32 
 12  City_H

,BHK,Rent,Size,Floor,Bathroom,Total Floors,Area Type_Built Area,Area Type_Carpet Area,Area Type_Super Area,City_Bangalore,...,City_Mumbai,Furnishing Status_Furnished,Furnishing Status_Semi-Furnished,Furnishing Status_Unfurnished,Tenant Preferred_Bachelors,Tenant Preferred_Bachelors/Family,Tenant Preferred_Family,Point of Contact_Contact Agent,Point of Contact_Contact Builder,Point of Contact_Contact Owner
0,2,10000,1100,0,2,2,0,0,1,0,...,0,0,0,1,0,1,0,0,0,1
1,2,20000,800,1,1,3,0,0,1,0,...,0,0,1,0,0,1,0,0,0,1
2,2,17000,1000,1,1,3,0,0,1,0,...,0,0,1,0,0,1,0,0,0,1
3,2,10000,800,1,1,2,0,0,1,0,...,0,0,0,1,0,1,0,0,0,1
4,2,7500,850,1,1,2,0,1,0,0,...,0,0,0,1,1,0,0,0,0,1


In [1177]:
data.corr()['Rent'].sort_values(ascending=False)

Rent                                 1.000000
Bathroom                             0.441215
Size                                 0.413551
BHK                                  0.369718
Total Floors                         0.352268
Point of Contact_Contact Agent       0.339750
City_Mumbai                          0.327038
Floor                                0.326200
Area Type_Carpet Area                0.215769
Furnishing Status_Furnished          0.110576
Tenant Preferred_Family              0.063941
Furnishing Status_Semi-Furnished     0.045309
Tenant Preferred_Bachelors           0.042151
Point of Contact_Contact Builder    -0.005482
Area Type_Built Area                -0.006439
City_Delhi                          -0.027072
City_Bangalore                      -0.061512
Tenant Preferred_Bachelors/Family   -0.078774
City_Chennai                        -0.082361
City_Hyderabad                      -0.087465
City_Kolkata                        -0.105322
Furnishing Status_Unfurnished     

### Разбиваем данные и масштабируем данные

In [1178]:
# Разделение на признаки и целевую переменную
X = data.drop('Rent', axis=1)
y = data['Rent']

# Разделение на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [1179]:
# Масштабируем данные
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

### Получить решение задачи в замкнутом виде с помощью псевдообратной матрицы. Рассмотреть случаи без регуляризации и с 𝐿 регуляризацией.
Для этого воспользуемся функциями из numpy, а позже получим те же значения с помощью функций из sklearn для сравнения



Нормальное уравнение линейной регрессии: <math xmlns="http://www.w3.org/1998/Math/MathML">
  <mi>w</mi>
  <mo>=</mo>
  <msup>
    <mi>X</mi>
    <mi>T</mi>
  </msup>
  <mo stretchy="false">(</mo>
  <msup>
    <mi>X</mi>
    <mi>T</mi>
  </msup>
  <mi>X</mi>
  <msup>
    <mo stretchy="false">)</mo>
    <mrow data-mjx-texclass="ORD">
      <mo>&#x2212;</mo>
      <mn>1</mn>
    </mrow>
  </msup>
  <mi>y</mi>
</math>

In [1180]:
# Найдем решение с помощью псевдообратной матрицы для OLS
def OLS(X, y):
    # добавляем столбец единиц для нахождения свободного члена
    X = np.hstack((X, np.ones((len(X), 1))))
    X_pinv = np.linalg.pinv(X)
    # Нахождение весов через псевдообратную матрицу
    weights = np.dot(X_pinv, y).reshape(-1)
    intercept = weights[-1]
    weights = weights[:-1]
    return weights, intercept

ols_weights, ols_intercept = OLS(X_train, y_train)

print('weights:', ols_weights)
print('intercept term:', ols_intercept)

weights: [ 2721.53917778 23138.64805111  5752.83069792  8897.38235365
  2521.76673586   131.3452297    783.12319341  -789.12777744
 -1853.97340855 -4662.11295199  1164.81642769 -8288.01261095
 -2043.19877131 14848.8007736   2511.51783559 -1183.83968582
  -585.62414663  1123.39631449   905.58874774 -2793.12813985
  1552.50734069   484.59788912 -1569.00288163]
intercept term: 35151.516332982064


Нормальное уравнение линейной регрессии с L2-регуляризацией: <math xmlns="http://www.w3.org/1998/Math/MathML">
  <mi>w</mi>
  <mo>=</mo>
  <mo stretchy="false">(</mo>
  <msup>
    <mi>X</mi>
    <mi>T</mi>
  </msup>
  <mi>X</mi>
  <mo>+</mo>
  <mi>&#x3B1;</mi>
  <mi>I</mi>
  <msup>
    <mo stretchy="false">)</mo>
    <mrow data-mjx-texclass="ORD">
      <mo>&#x2212;</mo>
      <mn>1</mn>
    </mrow>
  </msup>
  <msup>
    <mi>X</mi>
    <mi>T</mi>
  </msup>
  <mi>y</mi>
</math>

где <math xmlns="http://www.w3.org/1998/Math/MathML"><mi>&#x3B1;</mi></math> - коэффициент регуляризации, <math xmlns="http://www.w3.org/1998/Math/MathML"><mi>I</mi></math> - единичная матрица.

In [1181]:
# Найдем решение с помощью псевдообратной матрицы для случая линейной регрессии c L2-регуляризацией
def ridge(X, y, alpha):
    # добавляем столбец единиц для нахождения свободного члена
    X = np.hstack((X, np.ones((len(X), 1))))
    I = np.identity(len(X[0]))
    X_inv = np.linalg.inv(np.dot(X.T, X) + alpha*I)
    X_pinv = np.dot(X_inv, X.T)
    # Нахождение весов через псевдообратную матрицу
    weights = np.dot(X_pinv, y).reshape(-1)
    intercept = weights[-1]
    weights = weights[:-1]
    return weights, intercept

ridge_weights, ridge_intercept = ridge(X_train, y_train, alpha=0.001)
print('weights:', ridge_weights)
print('intercept term:', ridge_intercept)

weights: [ 2721.54584516 23138.63200596  5752.82732344  8897.38489772
  2521.77239251   131.3451221    783.12327632  -789.12790587
 -1853.97145193 -4662.11051319  1164.81472914 -8288.00821449
 -2043.19787468 14848.79299641  2511.51835025 -1183.83944329
  -585.6247682   1123.39589272   905.58851104 -2793.12720282
  1552.50974018   484.5975515  -1569.00526694]
intercept term: 35151.50707283794


In [1182]:
def gradient_descent(X, y, theta_init, learning_rate, num_iterations):
    theta = theta_init.copy()
    m = len(y)
    for _ in range(num_iterations):
        error = np.dot(X, theta) - y
        gradient = (1/m) * np.dot(X.T, error)
        theta = theta - learning_rate * gradient  # Reshape the gradient array
    intercept = theta[0]
    weights = theta[1:]
    return weights, intercept

X_train_with_bias = np.c_[np.ones((X_train.shape[0], 1)), X_train]
X_test_with_bias = np.c_[np.ones((X_test.shape[0], 1)), X_test]

theta_init = np.zeros(X_train_with_bias.shape[1])  # Инициализируем нулями
learning_rate = 0.01 
num_iterations = 1000  

gd_weights, gd_intercept = gradient_descent(X_train_with_bias, y_train, theta_init, learning_rate, num_iterations)

print('weights:', gd_weights)
print('intercept term:', gd_intercept)

weights: [ 3287.30127843 21948.81102828  5163.59323822  9356.27238342
  3217.2618016    136.80105257   744.81415653  -751.07056067
 -1696.47197865 -4565.18246621  1040.6926467  -8116.26768625
 -1976.21325183 14488.62999508  2566.72545424 -1187.71032127
  -621.24301889  1138.86545415   900.97124451 -2805.98735904
  1667.06168271   478.2682147  -1683.31281693]
intercept term: 35149.9987981736


In [1183]:
def print_metrics(y_test_pred):
    # Evaluation metrics
    mae = metrics.mean_absolute_error(y_test, y_test_pred)
    mse=  metrics.mean_squared_error(y_test, y_test_pred)
    rmse =  np.sqrt(mse)
    mape = metrics.mean_absolute_percentage_error(y_test, y_test_pred)
    r2 = metrics.r2_score(y_test, y_test_pred)
    print('MAE:', round(mae, 4))
    print('MSE:', round(mse, 4) )
    print('RMSE:', round(rmse, 4))
    print('MAPE:',round(mape, 4))
    print('r^2:', round(r2, 4))
    print('\n')

print("Линейная регрессия")
print_metrics(np.dot(X_test, ols_weights) + ols_intercept)
print("Ridge (L2)")
print_metrics(np.dot(X_test, ridge_weights) + ols_intercept)
print("Градиентный спуск")
print_metrics(np.dot(X_test, gd_weights) + gd_intercept)

Линейная регрессия
MAE: 21777.3445
MSE: 1908236404.5231
RMSE: 43683.3653
MAPE: 1.1078
r^2: 0.5212


Ridge (L2)
MAE: 21777.3417
MSE: 1908236508.6655
RMSE: 43683.3665
MAPE: 1.1078
r^2: 0.5212


Градиентный спуск
MAE: 21689.188
MSE: 1917285475.2442
RMSE: 43786.8185
MAPE: 1.1018
r^2: 0.5189




In [1184]:
# для проверки себя посчитаем то же самое с помощью sklearn

# Модель линейной регрессии без регуляризации
ols_linear_reg = LinearRegression()
ols_linear_reg.fit(X_train, y_train)
ols_linear_reg_prediction = ols_linear_reg.predict(X_test)

print("Значения функции потерь (линейная регрессия):")
print_metrics(ols_linear_reg_prediction)

# Модель линейной регрессии с L2-регуляризацией (Ridge)
ridge_reg = Ridge(alpha=0.5)
ridge_reg.fit(X_train, y_train)
ridge_reg_prediction = ridge_reg.predict(X_test)
print("Значения функции потерь (регуляризация Ridge):")
print_metrics(ridge_reg_prediction)

Значения функции потерь (линейная регрессия):
MAE: 21706.5913
MSE: 1908284470.189
RMSE: 43683.9155
MAPE: 1.0987
r^2: 0.5212


Значения функции потерь (регуляризация Ridge):
MAE: 21775.9024
MSE: 1908288474.9554
RMSE: 43683.9613
MAPE: 1.1076
r^2: 0.5212




### Какая модель имеет наименьшее значение функции потерь на тестовой выборке?
Наименьшее значение функции потерь MSE на тестовой выборке имеет модель, полученная градиентным спуском, следовательно, она имеет наилучшую точность

Значение функции потерь для случая с L2 регуляцией меньше, чем без него, но **крайне** незначительно. 

В целом, использование L2-регуляризации (Ridge) позволяет штрафовать большие значения коэффициентов модели, что помогает уменьшить переобучение, но в нашем случае регуляция не внесла значительного влияния на результаты.

Получим модель линейной регрессии с помощью стохастического градиентного спуска

### Какие признаки оказывают наибольший вклад в точность определения стоимости аренды?

Чтобы определить признаки, которые оказывают наибольший вклад в точность определения стоимости аренды, можно использовать коэффициенты модели линейной регрессии. Чем больше абсолютное значение коэффициента, тем больший вклад вносит соответствующий признак. Отранжируем признаки от наиболее значимого к наименее значимому.

Получается, что самыми важными признаками являются 
- площадь помещения, 
- город расположения, 
- количество ванных комнат,
- этаж, на котором располагаются аппартаменты.

In [1185]:
# Выведем все коэффициенты от самого высокого по модулю до самого низкого (т.е. от более значимых до менее значимых) для каждой из представленных моделей
feature_importance = pd.DataFrame({"Признак": X.columns, "Коэффициент": ols_weights})
feature_importance_sorted = feature_importance.iloc[feature_importance['Коэффициент'].abs().argsort()[::-1]]
print("Наиболее важные признаки для линейной регрессии:")
print(feature_importance_sorted)

Наиболее важные признаки для линейной регрессии:
                              Признак   Коэффициент
1                                Size  23138.648051
13                        City_Mumbai  14848.800774
3                            Bathroom   8897.382354
11                     City_Hyderabad  -8288.012611
2                               Floor   5752.830698
9                        City_Chennai  -4662.112952
19            Tenant Preferred_Family  -2793.128140
0                                 BHK   2721.539178
4                        Total Floors   2521.766736
14        Furnishing Status_Furnished   2511.517836
12                       City_Kolkata  -2043.198771
8                      City_Bangalore  -1853.973409
22     Point of Contact_Contact Owner  -1569.002882
20     Point of Contact_Contact Agent   1552.507341
15   Furnishing Status_Semi-Furnished  -1183.839686
10                         City_Delhi   1164.816428
17         Tenant Preferred_Bachelors   1123.396314
18  Tenant Pref

In [1186]:
feature_importance = pd.DataFrame({"Признак": X.columns, "Коэффициент": ridge_weights})
feature_importance_sorted = feature_importance.iloc[feature_importance['Коэффициент'].abs().argsort()[::-1]]
print("Наиболее важные признаки для Ridge:")
print(feature_importance_sorted)

Наиболее важные признаки для Ridge:
                              Признак   Коэффициент
1                                Size  23138.632006
13                        City_Mumbai  14848.792996
3                            Bathroom   8897.384898
11                     City_Hyderabad  -8288.008214
2                               Floor   5752.827323
9                        City_Chennai  -4662.110513
19            Tenant Preferred_Family  -2793.127203
0                                 BHK   2721.545845
4                        Total Floors   2521.772393
14        Furnishing Status_Furnished   2511.518350
12                       City_Kolkata  -2043.197875
8                      City_Bangalore  -1853.971452
22     Point of Contact_Contact Owner  -1569.005267
20     Point of Contact_Contact Agent   1552.509740
15   Furnishing Status_Semi-Furnished  -1183.839443
10                         City_Delhi   1164.814729
17         Tenant Preferred_Bachelors   1123.395893
18  Tenant Preferred_Bachelo

In [1187]:
feature_importance = pd.DataFrame({"Признак": X.columns, "Коэффициент": gd_weights})
feature_importance_sorted = feature_importance.iloc[feature_importance['Коэффициент'].abs().argsort()[::-1]]
print("Наиболее важные признаки для градиентного спуска:")
print(feature_importance_sorted)

Наиболее важные признаки для градиентного спуска:
                              Признак   Коэффициент
1                                Size  21948.811028
13                        City_Mumbai  14488.629995
3                            Bathroom   9356.272383
11                     City_Hyderabad  -8116.267686
2                               Floor   5163.593238
9                        City_Chennai  -4565.182466
0                                 BHK   3287.301278
4                        Total Floors   3217.261802
19            Tenant Preferred_Family  -2805.987359
14        Furnishing Status_Furnished   2566.725454
12                       City_Kolkata  -1976.213252
8                      City_Bangalore  -1696.471979
22     Point of Contact_Contact Owner  -1683.312817
20     Point of Contact_Contact Agent   1667.061683
15   Furnishing Status_Semi-Furnished  -1187.710321
17         Tenant Preferred_Bachelors   1138.865454
10                         City_Delhi   1040.692647
18  Tenant Pre